In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from surrogate import ImageSurrogate

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

In [4]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [5]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

## Load Data

In [6]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [7]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


### Batch Data

In [8]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [9]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Model

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(10, activation='softmax')(net)

model = Model(model_input, out)

model_weights_path = 'model/20210514_14_39_21/model_weights.h5'

model.load_weights(model_weights_path)
model.trainable = False

# Train Surrogate

### Save Dir

In [11]:
date = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'surrogate'
model_dir = os.path.join(os.getcwd(), save_dir, date)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Train

In [12]:
surrogate = ImageSurrogate(model, model_dir)

t = time.time()
surrogate.train(train_data = ds_train, 
                val_data = ds_val, 
                max_epochs = 100, 
                batch_size = 32, 
                lookback = 20,
                lr = 1e-3)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/100
1563

1563/1563 [==============================] - 227s 145ms/step - loss: 0.6142 - accuracy: 0.7893 - val_loss: 0.8484 - val_accuracy: 0.7304
Epoch 21/100
1563/1563 [==============================] - ETA: 0s - loss: 0.6146 - accuracy: 0.7906
Epoch 00021: val_loss improved from 0.84843 to 0.83876, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/surrogate/20210515_01_11_10/value_weights.h5
1563/1563 [==============================] - 227s 145ms/step - loss: 0.6146 - accuracy: 0.7906 - val_loss: 0.8388 - val_accuracy: 0.7268
Epoch 22/100
1563/1563 [==============================] - ETA: 0s - loss: 0.6032 - accuracy: 0.7957
Epoch 00022: val_loss did not improve from 0.83876
1563/1563 [==============================] - 227s 145ms/step - loss: 0.6032 - accuracy: 0.7957 - val_loss: 0.8724 - val_accuracy: 0.7176
Epoch 23/100
1563/1563 [==============================] - ETA: 0s - loss: 0.6027 - accuracy: 0.7931
Epoch 00023: val_loss did not improve from 0.83876
1563/1563

Epoch 45/100
1563/1563 [==============================] - ETA: 0s - loss: 0.5335 - accuracy: 0.8185
Epoch 00045: val_loss did not improve from 0.78711

Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.0004304672533180565.
1563/1563 [==============================] - 227s 145ms/step - loss: 0.5335 - accuracy: 0.8185 - val_loss: 0.8111 - val_accuracy: 0.7402
Epoch 46/100
1563/1563 [==============================] - ETA: 0s - loss: 0.5203 - accuracy: 0.8262
Epoch 00046: val_loss improved from 0.78711 to 0.77353, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/surrogate/20210515_01_11_10/value_weights.h5
1563/1563 [==============================] - 227s 145ms/step - loss: 0.5203 - accuracy: 0.8262 - val_loss: 0.7735 - val_accuracy: 0.7498
Epoch 47/100
1563/1563 [==============================] - ETA: 0s - loss: 0.5269 - accuracy: 0.8204
Epoch 00047: val_loss did not improve from 0.77353
1563/1563 [==============================] - 227s 145ms/step - lo

Epoch 70/100
1563/1563 [==============================] - ETA: 0s - loss: 0.4803 - accuracy: 0.8383
Epoch 00070: val_loss did not improve from 0.73836

Epoch 00070: ReduceLROnPlateau reducing learning rate to 0.00025418660952709616.
1563/1563 [==============================] - 227s 145ms/step - loss: 0.4803 - accuracy: 0.8383 - val_loss: 0.7908 - val_accuracy: 0.7478
Epoch 71/100
1563/1563 [==============================] - ETA: 0s - loss: 0.4862 - accuracy: 0.8349
Epoch 00071: val_loss did not improve from 0.73836
1563/1563 [==============================] - 227s 145ms/step - loss: 0.4862 - accuracy: 0.8349 - val_loss: 0.7761 - val_accuracy: 0.7512
Epoch 72/100
1563/1563 [==============================] - ETA: 0s - loss: 0.4899 - accuracy: 0.8353
Epoch 00072: val_loss did not improve from 0.73836
1563/1563 [==============================] - 227s 145ms/step - loss: 0.4899 - accuracy: 0.8353 - val_loss: 0.7930 - val_accuracy: 0.7444
Epoch 73/100
1563/1563 [==============================